# Enviroment Setup

In [1]:
!pip install opencv-python-headless
import os
import cv2
from tqdm import tqdm
from ipywidgets import interact, widgets
from IPython.display import display, HTML
from base64 import b64encode
from moviepy.editor import VideoFileClip
import subprocess


!git clone -b training https://github.com/YUyou0/ict3104-team14-2023.git

!sudo apt-get install python3.8
!sudo apt-get install python3.8-distutils
!sudo apt-get update
!python --version
!apt-get update
!apt install software-properties-common
!sudo dpkg --remove --force-remove-reinstreq python3-pip python3-setuptools python3-wheel
!apt-get install python3-pip

!cd /content/ict3104-team14-2023
!export PYTHONPATH=/content/FollowYourPose:$PYTHONPATH
# %pip install -r requirements.txt
!python3.8 -m pip install -q -U --pre triton
!apt update
!python3.8 -m pip install -q diffusers==0.11.1 torch==1.13.1 transformers==4.26.0 bitsandbytes==0.35.4 \
imageio-ffmpeg xformers==0.0.16  --extra-index-url https://download.pytorch.org/whl/cu113

%cd /content/ict3104-team14-2023/
!python3.8 -m pip install -r requirements.txt

Cloning into 'ict3104-team14-2023'...
remote: Enumerating objects: 143, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 143 (delta 10), reused 37 (delta 8), pack-reused 104
Receiving objects: 100% (143/143), 67.31 MiB | 39.98 MiB/s, done.
Resolving deltas: 100% (44/44), done.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libpython3.8-minimal libpython3.8-stdlib mailcap mime-support
  python3.8-minimal
Suggested packages:
  python3.8-venv binfmt-support
The following NEW packages will be installed:
  libpython3.8-minimal libpython3.8-stdlib mailcap mime-support python3.8
  python3.8-minimal
0 upgraded, 6 newly installed, 0 to remove and 18 not upgraded.
Need to get 5,098 kB of archives.
After this operation, 18.9 MB of additional disk space will be used.
Get:1 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu ja

In [2]:
%mkdir checkpoints
!git lfs install
!git clone https://huggingface.co/YueMafighting/FollowYourPose_v1
%mv /content/ict3104-team14-2023/FollowYourPose_v1/* /content/ict3104-team14-2023/checkpoints/
%rm -rf FollowYourPose_v1

!sudo apt-get install python3.8
!sudo apt-get install python3.8-distutils

Updated git hooks.
Git LFS initialized.
Cloning into 'FollowYourPose_v1'...
remote: Enumerating objects: 42, done.
remote: Total 42 (delta 0), reused 0 (delta 0), pack-reused 42
Unpacking objects: 100% (42/42), 584.62 KiB | 5.04 MiB/s, done.
Filtering content: 100% (9/9), 9.75 GiB | 52.29 MiB/s, done.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
python3.8 is already the newest version (3.8.18-1+jammy1).
0 upgraded, 0 newly installed, 0 to remove and 25 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
python3.8-distutils is already the newest version (3.8.18-1+jammy1).
0 upgraded, 0 newly installed, 0 to remove and 25 not upgraded.


# Select a pretrained model that you want to use for training

In [ ]:
# Define the path to the folder containing the files
folder_path = '/content/ict3104-team14-2023/configs'  # Replace with the actual folder path

# List files in the folder
model_list = os.listdir(folder_path)
model_list.insert(0,"")  # Insert "" at the start of the list

# Create a dropdown widget
model_dropdown = widgets.Dropdown(
    options=model_list,
    description='Select a file:'
)

# Display the dropdown widget
display(model_dropdown)

Dropdown(description='Select a file:', options=('', 'pose_sample.yaml'), value='')

# Run this cell below to confirm your selected pretrained model to be used for training

In [ ]:
# Get the selected file name from the dropdown
selected_model = model_dropdown.value

# Check if the selected option is valid
if selected_model == "":
    raise ValueError("Invalid option selected. Please choose from the provided list.")

# Now 'selected_file' contains the selected file name as a string
print(f"Selected model: {selected_model}")

Selected model: pose_sample.yaml


# Select a dataset to be used for the training

In [ ]:
import os
import ipywidgets as widgets
from IPython.display import display

# Function to list file names in a folder and store their paths in an array
def list_file_names_and_paths(folder):
    global file_paths
    folder_path = os.path.join('/content/ict3104-team14-2023/input_videos', folder)
    if os.path.isdir(folder_path):
        file_paths = [os.path.join(folder_path, file) for file in os.listdir(folder_path)]
        file_names = [os.path.basename(file) for file in file_paths]
        print(f"List of file names in '{folder}':")
        for file_name in file_names:
            print(file_name)
    else:
        print(f"'{folder}' is not a valid folder.")

# Create a dropdown widget with an empty default option
folder_dropdown = widgets.Dropdown(
    options=[''] + os.listdir('/content/ict3104-team14-2023/input_videos'),  # Include an empty option
    description='Select a Folder:',
)

# Initialize the file_paths array
file_paths = []

# Define an event handler for the dropdown
def on_dropdown_change(change):
    selected_folder = change.new
    list_file_names_and_paths(selected_folder)

# Attach the event handler to the dropdown
folder_dropdown.observe(on_dropdown_change, names='value')

# Display the dropdown widget
display(folder_dropdown)

Dropdown(description='Select a Folder:', options=('', 'charades'), value='')

List of file names in 'charades':
52CKM.mp4
3LMTS.mp4
5D3X6.mp4
5O0YS.mp4
3JJ7C.mp4
0RJKT.mp4
4GLAP.mp4
3IPI3.mp4
5EFQ7.mp4
8ASRQ.mp4
2E8GU.mp4


# List of videos to be used for training

In [ ]:
print(file_paths)
print(len(file_paths))

['/content/ict3104-team14-2023/input_videos/charades/52CKM.mp4', '/content/ict3104-team14-2023/input_videos/charades/3LMTS.mp4', '/content/ict3104-team14-2023/input_videos/charades/5D3X6.mp4', '/content/ict3104-team14-2023/input_videos/charades/5O0YS.mp4', '/content/ict3104-team14-2023/input_videos/charades/3JJ7C.mp4', '/content/ict3104-team14-2023/input_videos/charades/0RJKT.mp4', '/content/ict3104-team14-2023/input_videos/charades/4GLAP.mp4', '/content/ict3104-team14-2023/input_videos/charades/3IPI3.mp4', '/content/ict3104-team14-2023/input_videos/charades/5EFQ7.mp4', '/content/ict3104-team14-2023/input_videos/charades/8ASRQ.mp4', '/content/ict3104-team14-2023/input_videos/charades/2E8GU.mp4']
11


# Input a name for this new model to be trained

In [ ]:
# Create a text input widget
text_input = widgets.Text(
    value='',  # Default value (you can set a default value if needed)
    placeholder='Enter new model name',  # Placeholder text
    description='Name Input:',  # Label for the input field
)

# Create a submit button
submit_button = widgets.Button(
    description='Submit',  # Label for the button
)

# Create a variable to store the user's input
user_input_modelName = ''

# Define a function to update the user_input variable when the submit button is clicked
def on_submit_button_click(b):
    global user_input_modelName
    user_input_modelName = text_input.value
    if user_input_modelName == "":
      print("You have not created a name for this new model!")
    else:
      print("New model name has been submitted")
    if user_input_modelName.isalnum():
        print(f"Model name accepted: {user_input_modelName}")
    else:
        print("Invalid input. Please enter alphanumeric characters only (no spaces or special characters).")

# Attach the click event handler to the submit button
submit_button.on_click(on_submit_button_click)

# Display the widgets
display(text_input, submit_button)

Text(value='', description='Name Input:', placeholder='Enter new model name')

Button(description='Submit', style=ButtonStyle())

New model name has been submitted
Model name accepted: naruto


# Verify the newly named model

In [ ]:
print(user_input_modelName)

naruto


# Input the batch size

In [ ]:
# Function to handle the button click event
def on_submit_button_click(b):
    global user_input_batchSize
    user_input_batchSize = text_input.value
    # Check if the input is a valid whole number (integer)
    if user_input_batchSize.isdigit():
        print(f"Batch size accepted: {user_input_batchSize}")
    else:
        print("Invalid input. Please enter a whole number (no decimals).")

# Create a text input widget
text_input = widgets.Text(
    placeholder='Enter a whole number',
    description='Batch Size:',
)

# Create a submit button
submit_button = widgets.Button(
    description='Submit',
)
submit_button.on_click(on_submit_button_click)

# Display the input field and submit button
display(text_input)
display(submit_button)

Text(value='', description='Batch Size:', placeholder='Enter a whole number')

Button(description='Submit', style=ButtonStyle())

Invalid input. Please enter a whole number (no decimals).
Batch size accepted: 3


# Verify the batch size

In [ ]:
print(user_input_batchSize)

3


# Set epochs value

In [ ]:
# Function to handle the button click event
def on_submit_button_click(b):
    global user_input_epoch
    user_input_epoch = text_input.value
    # Check if the input is a valid whole number (integer)
    if user_input_epoch.isdigit():
        print(f"Epochs value accepted: {user_input_epoch}")
    else:
        print("Invalid input. Please enter a whole number (no decimals).")

# Create a text input widget
text_input = widgets.Text(
    placeholder='Enter a whole number',
    description='Epochs:',
)

# Create a submit button
submit_button = widgets.Button(
    description='Submit',
)
submit_button.on_click(on_submit_button_click)

# Display the input field and submit button
display(text_input)
display(submit_button)

Text(value='', description='Epochs:', placeholder='Enter a whole number')

Button(description='Submit', style=ButtonStyle())

Epochs value accepted: 4


#Verify epoch value

In [ ]:
print(user_input_epoch)

4


# Run the cell below for validation checking before starting the training sequence

In [ ]:
if selected_model != "":
  print("Pretrained model loaded successful")
  print(selected_model)

print("\n")

if file_paths != "":
   print("Dataset loaded successful")

print("\n")

if user_input_modelName != "":
   print(f"New model name: {user_input_modelName}")

print("\n")


if user_input_batchSize != "":
  print(f"Batch Size: {user_input_batchSize}")

print("\n")

if user_input_epoch != "":
  print(f"Epoch: {user_input_epoch}")

if selected_model == "":
    raise ValueError("Invalid option selected. Please choose from the provided list.")

# Check if the selected option is valid
if file_paths == "":
    raise ValueError("Invalid option selected. Please choose from the provided list.")

# Check if the selected option is valid
if user_input_batchSize == "":
    raise ValueError("You have not written a validation prompt.")


Pretrained model loaded successful
pose_sample.yaml


Dataset loaded successful


New model name: naruto


Batch Size: 3


Epoch: 4


# Start Training

In [10]:
%cd /content/ict3104-team14-2023/
!pwd

!TORCH_DISTRIBUTED_DEBUG=DETAIL accelerate launch \
    --gpu_ids '0'\
    train_followyourpose.py \
    --config="configs/pose_train.yaml"

/content/ict3104-team14-2023
/content/ict3104-team14-2023
The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `1`
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
A matching Triton is not available, some optimizations will not be enabled.
Error caught was: module 'triton.language' has no attribute 'constexpr'
/usr/local/lib/python3.8/dist-packages/torchvision/transforms/_functional_video.py:6: UserWarning: The 'torchvision.transforms._functional_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms.functional' module instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/transforms/_transforms_video.py:22: UserWarning: The 'torchvisi